# Importing Libraries

In [1]:
import mysql.connector
import pandas as pd

# Creating a reconnection function

In [2]:
def create_connection():
    """Create and return a MySQL database connection and cursor."""
    
    # Establish the connection
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="Library_Management_System"
    )
    
    # Create a cursor
    conn = mydb.cursor()
    
    print("Connection to the database was successful.")
    return mydb, conn

# Coding the database

## Creating connection object

In [3]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database=""
)

print(mydb)

## Creating the cursor

In [4]:
conn = mydb.cursor()

## Creating the MySQL code

In [5]:
conn.execute("DROP DATABASE IF EXISTS Library_Management_System")

conn.execute("CREATE DATABASE IF NOT EXISTS Library_Management_System")

conn.execute("USE Library_Management_System")

create_sql =  """
CREATE TABLE IF NOT EXISTS Subjects (
    SubjectID INT AUTO_INCREMENT PRIMARY KEY,
    SubjectName VARCHAR(255),
    Description TEXT
);

CREATE TABLE IF NOT EXISTS Authors (
    AuthorID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(255),
    Affiliation VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS UserTypes (
    UserTypeID INT AUTO_INCREMENT PRIMARY KEY,
    TypeName VARCHAR(50) NOT NULL,
    Accesslevel VARCHAR(50) NOT NULL,
    BorrowingLimit INT NOT NULL
);

CREATE TABLE IF NOT EXISTS Users (
    UserID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(255),
    Email VARCHAR(255),
    Department VARCHAR(255),
    Affiliation VARCHAR(255),
    UserTypeID INT,
    FOREIGN KEY (UserTypeID) REFERENCES UserTypes(UserTypeID)
);

CREATE TABLE IF NOT EXISTS Resources (
    ResourceID INT AUTO_INCREMENT PRIMARY KEY,
    Title VARCHAR(255),
    PublicationDate DATE,
    Publisher VARCHAR(255),
    ISBN VARCHAR(13),
    Format ENUM('Physical', 'Digital'),
    SubjectID INT,
    AuthorID INT,
    FOREIGN KEY (SubjectID) REFERENCES Subjects(SubjectID),
    FOREIGN KEY (AuthorID) REFERENCES Authors(AuthorID)
);

CREATE TABLE IF NOT EXISTS BorrowingRecords (
    BorrowingRecordID INT AUTO_INCREMENT PRIMARY KEY,
    UserID INT,
    ResourceID INT,
    BorrowedDate DATE,
    DueDate DATE,
    ReturnedDate DATE,
    FOREIGN KEY (UserID) REFERENCES Users(UserID),
    FOREIGN KEY (ResourceID) REFERENCES Resources(ResourceID)
);

CREATE TABLE IF NOT EXISTS Fines (
    FineID INT AUTO_INCREMENT PRIMARY KEY,
    BorrowingRecordID INT,
    FineAmount DECIMAL(10, 2),
    PaymentDate DATE,
    FOREIGN KEY (BorrowingRecordID) REFERENCES BorrowingRecords(BorrowingRecordID)
);

CREATE TABLE IF NOT EXISTS Reservations (
    ReservationID INT AUTO_INCREMENT PRIMARY KEY,
    UserID INT,
    ResourceID INT,
    ReservationDate DATE,
    Status ENUM('Pending', 'Approved', 'Denied'),
    FOREIGN KEY (UserID) REFERENCES Users(UserID),
    FOREIGN KEY (ResourceID) REFERENCES Resources(ResourceID)
);

CREATE TABLE IF NOT EXISTS Reviews (
    ReviewID INT AUTO_INCREMENT PRIMARY KEY,
    UserID INT,
    ResourceID INT,
    Rating INT,
    ReviewText TEXT,
    FOREIGN KEY (UserID) REFERENCES Users(UserID),
    FOREIGN KEY (ResourceID) REFERENCES Resources(ResourceID)
);

CREATE TABLE IF NOT EXISTS Keywords (
    KeywordID INT AUTO_INCREMENT PRIMARY KEY,
    KeywordText VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS ResourceKeywords (
    ResourceID INT,
    KeywordID INT,
    PRIMARY KEY (ResourceID, KeywordID),
    FOREIGN KEY (ResourceID) REFERENCES Resources(ResourceID),
    FOREIGN KEY (KeywordID) REFERENCES Keywords(KeywordID)
);
"""

conn.execute(create_sql,multi=True)
conn.close()

True

## Advanced Functionalities:

### Track resource availability (physical vs. digital copies, borrowing status)

In [6]:
mydb, conn = create_connection()
track_availability = """

CREATE PROCEDURE AddAvailableCopiesColumn()
BEGIN
    ALTER TABLE Resources
    ADD AvailableCopies INT DEFAULT 0;
END;
"""    
conn.execute(track_availability)

Connection to the database was successful.


### Calculate overdue fines based on resource type and borrowing duration

In [7]:
mydb, conn = create_connection()

calculating_overdue = """

CREATE PROCEDURE GetOverdueRecords()
BEGIN
    SELECT
        br.BorrowingRecordID,
        r.Title,
        br.DueDate,
        COALESCE(br.ReturnedDate, CURDATE()) AS ActualReturnDate,
        DATEDIFF(COALESCE(br.ReturnedDate, CURDATE()), br.DueDate) AS OverdueDays,
        CASE
            WHEN r.Format = 'Physical' THEN DATEDIFF(COALESCE(br.ReturnedDate, CURDATE()), br.DueDate) * 2
            WHEN r.Format = 'Digital' THEN DATEDIFF(COALESCE(br.ReturnedDate, CURDATE()), br.DueDate) * 1
            ELSE 0
        END AS TotalFine
    FROM 
        BorrowingRecords br
    JOIN 
        Resources r ON br.ResourceID = r.ResourceID
    WHERE 
        COALESCE(br.ReturnedDate, CURDATE()) > br.DueDate;
END;
"""

conn.execute(calculating_overdue)

Connection to the database was successful.


### Implement keyword search function for resource discovery

In [8]:
mydb,conn = create_connection()

resource_search = """

CREATE PROCEDURE SearchResources(IN keyword VARCHAR(255))
BEGIN
    SELECT *
    FROM RESOURCES
    WHERE Title LIKE CONCAT('%', keyword, '%') 
        OR Publisher LIKE CONCAT('%', keyword, '%') 
        OR ISBN LIKE CONCAT('%', keyword, '%');
END;
"""
conn.execute(resource_search)

Connection to the database was successful.


## Populating the data

### Inserting the values to the sql

#### Inserting Values to Subject

In [9]:
df_subject = pd.read_excel("University_Database.xlsx",sheet_name="Subject")
df_subject.head()

,SubjectID,SubjectName,Description
0,1,Computer Science,"Covers topics like AI, Machine Learning, etc."
1,2,Mathematics,"Includes Calculus, Algebra, etc."
2,3,Literature,Explores various literary genres and authors.
3,4,Science Fiction,Focuses on stories set in the future.


In [10]:
insert_SQL="""INSERT INTO Subjects (SubjectID ,SubjectName,Description)
VALUES (%s,%s,%s)"""
for _,row in df_subject.iterrows():
    conn.execute(insert_SQL,(row['SubjectID'],
                            row['SubjectName'],
                            row['Description']))
                        
mydb.commit()

In [11]:
query_subject=""" 
SELECT * FROM Subjects
"""
conn.execute(query_subject)

for row in conn:
    print(row)

(1, 'Computer Science', 'Covers topics like AI, Machine Learning, etc.')
(2, 'Mathematics', 'Includes Calculus, Algebra, etc.')
(3, 'Literature', 'Explores various literary genres and authors.')
(4, 'Science Fiction', 'Focuses on stories set in the future.')


#### Inserting values to Authors

In [12]:
df_authors = pd.read_excel("University_Database.xlsx",sheet_name="Authors")
df_authors.head()

,AuthorID,Name,Affiliation
0,1,John Smith,MIT
1,2,Jane Doe,Stanford
2,3,Michael Brown,Harvard
3,4,William Shakespeare,Public Domain
4,5,Christopher Marlowe,Public Domain


In [13]:
insert_SQL="""INSERT INTO Authors (AuthorID,Name,Affiliation)
VALUES (%s,%s,%s)"""
for _,row in df_authors.iterrows():
    conn.execute(insert_SQL,(row['AuthorID'],
                            row['Name'],
                            row['Affiliation']))
                        
mydb.commit()

In [14]:
query_Authors=""" 
SELECT * FROM Authors
"""
conn.execute(query_Authors)

for row in conn:
    print(row)

(1, 'John Smith', 'MIT')
(2, 'Jane Doe', 'Stanford')
(3, 'Michael Brown', 'Harvard')
(4, 'William Shakespeare', 'Public Domain')
(5, 'Christopher Marlowe', 'Public Domain')
(6, 'Ray Bradbury', 'Public Domain')


#### Inserting values in UserTypes

In [15]:
df_user_type = pd.read_excel("University_Database.xlsx",sheet_name="User_Type")
df_user_type.head()

,UserTypeID,TypeName,Accesslevel,BorrowingLimit
0,1,Student,Basic,3
1,2,Faculty,Extended,5
2,3,Staff,Basic,2


In [16]:
insert_SQL="""INSERT INTO UserTypes (UserTypeID,TypeName,Accesslevel,BorrowingLimit)
VALUES (%s,%s,%s,%s)"""
for _,row in df_user_type.iterrows():
    conn.execute(insert_SQL,(row['UserTypeID'],
                            row['TypeName'],
                            row['Accesslevel'],
                            row['BorrowingLimit']))
                        
mydb.commit()

In [17]:
query_user_types=""" 
SELECT * FROM UserTypes
"""
conn.execute(query_user_types)

for row in conn:
    print(row)

(1, 'Student', 'Basic', 3)
(2, 'Faculty', 'Extended', 5)
(3, 'Staff', 'Basic', 2)


#### Inserting values in Users

In [18]:
df_user = pd.read_excel("University_Database.xlsx",sheet_name="Users")
df_user.head()

,UserID,Name,Email,Department,UserTypeID
0,1,Alice Student,[email address removed],Computer Science,1
1,2,Bob Faculty,[email address removed],Engineering,2
2,3,Charlie Staff,[email address removed],Administration,3


In [19]:
insert_SQL="""INSERT INTO Users (UserID,Name,Email,Department,UserTypeID)
VALUES (%s,%s,%s,%s,%s)"""
for _,row in df_user.iterrows():
    conn.execute(insert_SQL,(row['UserID'],
                            row['Name'],
                            row['Email'],
                            row['Department'],
                            row['UserTypeID']))
                        
mydb.commit()

In [20]:
query_users=""" 
SELECT * FROM Users
"""
conn.execute(query_users)

for row in conn:
    print(row)

(1, 'Alice Student', '[email address removed]', 'Computer Science', None, 1)
(2, 'Bob Faculty', '[email address removed]', 'Engineering', None, 2)
(3, 'Charlie Staff', '[email address removed]', 'Administration', None, 3)


#### Inserting values into Resources

In [21]:
df_Resources = pd.read_excel("University_Database.xlsx",sheet_name="Resources")
df_Resources.head()

,ResourceID,Title,AuthorID,SubjectID,PublicationDate,Publisher,ISBN,Format
0,1,Intro to AI,1,1,2023-01-01 00:00:00,Tech Publications,978-1234567890,Physical
1,2,Machine Learning Basics,2,1,2022-05-10 00:00:00,O'Reilly Media,978-0987654321,Digital
2,3,Calculus for Dummies,3,2,2020-12-31 00:00:00,Wiley,NaN,Physical
3,4,Shakespeare's Sonnets,4,3,1609-01-01,Public Domain,NaN,Physical
4,5,Shakespeare's Sonnets,5,3,1609-01-01,Public Domain,NaN,Physical


In [22]:
df_Resources.fillna("[Has No value]",inplace=True)
df_Resources.head()

,ResourceID,Title,AuthorID,SubjectID,PublicationDate,Publisher,ISBN,Format
0,1,Intro to AI,1,1,2023-01-01 00:00:00,Tech Publications,978-1234567890,Physical
1,2,Machine Learning Basics,2,1,2022-05-10 00:00:00,O'Reilly Media,978-0987654321,Digital
2,3,Calculus for Dummies,3,2,2020-12-31 00:00:00,Wiley,[Has No value],Physical
3,4,Shakespeare's Sonnets,4,3,1609-01-01,Public Domain,[Has No value],Physical
4,5,Shakespeare's Sonnets,5,3,1609-01-01,Public Domain,[Has No value],Physical


In [23]:
insert_SQL="""INSERT INTO Resources (ResourceID,Title,AuthorID,SubjectID,PublicationDate,Publisher,ISBN,Format)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
for _,row in df_Resources.iterrows():
    conn.execute(insert_SQL,(row['ResourceID'],
                            row['Title'],
                            row['AuthorID'],
                            row['SubjectID'],
                            row['PublicationDate'],
                            row['Publisher'],
                            row['ISBN'],
                            row['Format'],))
                        
mydb.commit()

In [24]:
query_Resources=""" 
SELECT * FROM Resources
"""
conn.execute(query_Resources)

for row in conn:
    print(row)

(1, 'Intro to AI', datetime.date(2023, 1, 1), 'Tech Publications', '978-123456789', 'Physical', 1, 1)
(2, 'Machine Learning Basics', datetime.date(2022, 5, 10), "O'Reilly Media", '978-098765432', 'Digital', 1, 2)
(3, 'Calculus for Dummies', datetime.date(2020, 12, 31), 'Wiley', '[Has No value', 'Physical', 2, 3)
(4, "Shakespeare's Sonnets", datetime.date(1609, 1, 1), 'Public Domain', '[Has No value', 'Physical', 3, 4)
(5, "Shakespeare's Sonnets", datetime.date(1609, 1, 1), 'Public Domain', '[Has No value', 'Physical', 3, 5)
(6, 'The Martian Chronicles', datetime.date(1950, 10, 26), 'Bantam Books', '[Has No value', 'Digital', 4, 6)


#### Inserting values in BorrowingRecords

In [25]:
insert_borrowing_SQL = """INSERT INTO BorrowingRecords (UserID, ResourceID, BorrowedDate, DueDate, ReturnedDate)
VALUES (%s, %s, %s, %s, %s)"""

borrowing_data = [
    (1, 1, '2024-01-15', '2024-02-15', None),  
    (1, 2, '2024-01-20', '2024-02-20', '2024-02-10'), 
    (2, 3, '2024-01-18', '2024-02-18', None),  
    (3, 4, '2024-01-25', '2024-02-25', None),  
    (2, 5, '2024-01-30', '2024-02-28', '2024-02-27'),  
]

for record in borrowing_data:
    conn.execute(insert_borrowing_SQL, record)

mydb.commit()

In [26]:
query_borrowing = """ 
SELECT * FROM BorrowingRecords
"""
conn.execute(query_borrowing)

print("Borrowing Records:")
for row in conn:
    print(row)

Borrowing Records:
(1, 1, 1, datetime.date(2024, 1, 15), datetime.date(2024, 2, 15), None)
(2, 1, 2, datetime.date(2024, 1, 20), datetime.date(2024, 2, 20), datetime.date(2024, 2, 10))
(3, 2, 3, datetime.date(2024, 1, 18), datetime.date(2024, 2, 18), None)
(4, 3, 4, datetime.date(2024, 1, 25), datetime.date(2024, 2, 25), None)
(5, 2, 5, datetime.date(2024, 1, 30), datetime.date(2024, 2, 28), datetime.date(2024, 2, 27))


#### Inserting values in fines

In [27]:
insert_fine_SQL = """INSERT INTO Fines (BorrowingRecordID, FineAmount, PaymentDate)
VALUES (%s, %s, %s)"""

fine_data = [
    (1, 10.00, '2024-02-16'),  
    (2, 0.00, None),           
    (3, 5.00, '2024-02-20'),   
]

for record in fine_data:
    conn.execute(insert_fine_SQL, record)

mydb.commit()

In [28]:
query_fines = """ 
SELECT * FROM Fines
"""
conn.execute(query_fines)

print("\nFines Records:")
for row in conn:
    print(row)


Fines Records:
(1, 1, Decimal('10.00'), datetime.date(2024, 2, 16))
(2, 2, Decimal('0.00'), None)
(3, 3, Decimal('5.00'), datetime.date(2024, 2, 20))


#### Inserting values in Keywords

In [29]:
insert_keyword_SQL = """INSERT INTO Keywords (KeywordText)
VALUES (%s)"""

keyword_data = [
    ('AI'),
('Machine Learning'),
('Calculus'),
('Literature'),
('Science Fiction'),
]

for keyword in keyword_data:
    conn.execute(insert_keyword_SQL, (keyword,))  

mydb.commit()


In [30]:
query_keywords = """ 
SELECT * FROM Keywords
"""
conn.execute(query_keywords)

print("\nKeywords:")
for row in conn:
    print(row)


Keywords:
(1, 'AI')
(2, 'Machine Learning')
(3, 'Calculus')
(4, 'Literature')
(5, 'Science Fiction')


#### Inserting values in Resourceskeywords

In [31]:
resource_keywords_data = [
    (1, 1),  
    (2, 1),  
    (2, 2),  
    (3, 3),  
    (4, 4),  
    (5, 5),
]

insert_resource_keyword_SQL = """INSERT INTO ResourceKeywords (ResourceID, KeywordID)
VALUES (%s, %s)"""

for record in resource_keywords_data:
    conn.execute(insert_resource_keyword_SQL, record)

mydb.commit()

In [32]:
query_resource_keywords = """ 
SELECT * FROM ResourceKeywords
"""
conn.execute(query_resource_keywords)

print("\nResource-Keywords relationship:")
for row in conn:
    print(row)


Resource-Keywords relationship:
(1, 1)
(2, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 5)


#### Inserting values in Reviews

In [33]:
insert_review_SQL = """INSERT INTO Reviews (UserID, ResourceID, Rating, ReviewText)
VALUES (%s, %s, %s, %s)"""

review_data = [
    (1, 1, 5, 'An excellent introduction to AI! Highly recommend.'),
    (2, 2, 4, 'Great resource for beginners in machine learning.'),
    (3, 3, 3, 'Helpful for calculus, but lacks depth in some areas.'),
    (1, 4, 5, 'A classic work of literature! Timeless.'),
    (2, 5, 4, 'Interesting perspective on Shakespeare’s works.'),
    (3, 6, 2, 'Not as engaging as I expected.'),
]

for review in review_data:
    conn.execute(insert_review_SQL, review)

mydb.commit()

In [34]:
query_reviews = """ 
SELECT * FROM Reviews
"""
conn.execute(query_reviews)

print("\nReviews:")
for row in conn:
    print(row)


Reviews:
(1, 1, 1, 5, 'An excellent introduction to AI! Highly recommend.')
(2, 2, 2, 4, 'Great resource for beginners in machine learning.')
(3, 3, 3, 3, 'Helpful for calculus, but lacks depth in some areas.')
(4, 1, 4, 5, 'A classic work of literature! Timeless.')
(5, 2, 5, 4, 'Interesting perspective on Shakespeare’s works.')
(6, 3, 6, 2, 'Not as engaging as I expected.')


#### Inserting values in Reservations

In [35]:
insert_reservation_SQL = """INSERT INTO Reservations (UserID, ResourceID, ReservationDate, Status)
VALUES (%s, %s, %s, %s)"""

reservation_data = [
    (1, 1, '2024-01-15', 'Approved'),   
    (2, 2, '2024-01-10', 'Pending'),     
    (3, 3, '2024-01-20', 'Denied'),      
    (1, 4, '2024-01-18', 'Approved'),    
    (2, 5, '2024-01-25', 'Approved'),    
    (3, 6, '2024-01-22', 'Pending'),     
]

for record in reservation_data:
    conn.execute(insert_reservation_SQL, record)

mydb.commit()

In [36]:
query_reservations = """ 
SELECT * FROM Reservations
"""
conn.execute(query_reservations)

print("\nReviews:")
for row in conn:
    print(row)


Reviews:
(1, 1, 1, datetime.date(2024, 1, 15), 'Approved')
(2, 2, 2, datetime.date(2024, 1, 10), 'Pending')
(3, 3, 3, datetime.date(2024, 1, 20), 'Denied')
(4, 1, 4, datetime.date(2024, 1, 18), 'Approved')
(5, 2, 5, datetime.date(2024, 1, 25), 'Approved')
(6, 3, 6, datetime.date(2024, 1, 22), 'Pending')


#### A function that can Automate that Process

In [37]:
# def process_and_insert_data(file_path, sheet_name, create_connection):
#     mydb, conn = create_connection()

#     df = pd.read_excel(file_path, sheet_name=sheet_name)

#     df.fillna("[Has No value]", inplace=True)

#     column_names = []
    
#     print("Enter column names one by one. Type 'done' when finished:")
    
#     while True:
#         column_name = input("Enter column names one by one. Type 'done' when finished: ")
#         if column_name.lower() == "done":
#             break
#         column_names.append(column_name.strip())

#     placeholders = ', '.join(['%s'] * len(column_names)) 
#     insert_SQL = f"""INSERT INTO {sheet_name} ({', '.join(column_names)})
#                      VALUES ({placeholders})"""

#     for _, row in df.iterrows():
#         conn.execute(insert_SQL, tuple(row[column] for column in column_names))
    
#     mydb.commit()

#     query = f"SELECT * FROM {sheet_name}"
#     conn.execute(query)

#     for row in conn:
#         print(row)


# # Example usage
# file_path = input("Please Enter the Path of the file you want to populate data from: ")
# sheet_name = input("Enter the sheet name (e.g., Resources, Authors, Users, UserTypes, Subjects): ")
# process_and_insert_data(file_path, sheet_name, create_connection)


##### It is working perfectly for me I just highlighted it to not interfere with other codes

# Important Note:

##### I have written some of the following procedures in the previous sections. 
##### However, I plan to either improve upon them or present more refined versions below.
##### This will ensure that the code is both optimized and better structured for its intended purpose.

## Advanced SQL Queries

### Find all overdue resources for faculty members in the Engineering department who have borrowed more than 5 items in total.

In [38]:
mydb, conn = create_connection()
overdue_resources = """
SELECT Users.Name, Resources.Title, BorrowingRecords.DueDate
FROM BorrowingRecords
JOIN Users ON BorrowingRecords.UserID = Users.UserID
JOIN Resources ON BorrowingRecords.ResourceID = Resources.ResourceID
WHERE Users.UserTypeID = 2 -- Faculty
  AND BorrowingRecords.DueDate < CURDATE()  -- Overdue items
GROUP BY Users.UserID
HAVING COUNT(BorrowingRecords.BorrowingRecordID) > 5;
"""


conn.execute(overdue_resources)
results = conn.fetchall()

if results:
    for row in results:
        print(f"User: {row[0]}, Resource: {row[1]}, Due Date: {row[2]}")
else:
    print("There are no results")

Connection to the database was successful.
There are no results


### Identify the Top 3 Most Popular Resources

In [39]:
mydb, conn = create_connection()
top_popular_resources = """
SELECT Resources.Title, COUNT(BorrowingRecords.BorrowingRecordID) AS BorrowCount, 
       AVG(Reviews.Rating) AS AverageRating
FROM Resources
LEFT JOIN BorrowingRecords ON Resources.ResourceID = BorrowingRecords.ResourceID
LEFT JOIN Reviews ON Resources.ResourceID = Reviews.ResourceID
WHERE Resources.Format IN ('Physical', 'Digital')  
GROUP BY Resources.ResourceID
ORDER BY BorrowCount DESC, AverageRating DESC
LIMIT 3;
"""

conn.execute(top_popular_resources)
results = conn.fetchall()

for row in results:
    print(f"Resource: {row[0]}, Borrow Count: {row[1]}, Average Rating: {row[2]}")

Connection to the database was successful.
Resource: Shakespeare's Sonnets, Borrow Count: 1, Average Rating: 5.0000
Resource: Intro to AI, Borrow Count: 1, Average Rating: 5.0000
Resource: Shakespeare's Sonnets, Borrow Count: 1, Average Rating: 4.0000


### Generate a Report for Each Department

In [40]:
department_borrowing_report = """
SELECT Users.Department, COUNT(BorrowingRecords.BorrowingRecordID) AS TotalBorrowed, 
       SUM(Fines.FineAmount) AS TotalFines
FROM BorrowingRecords
JOIN Users ON BorrowingRecords.UserID = Users.UserID
LEFT JOIN Fines ON BorrowingRecords.BorrowingRecordID = Fines.BorrowingRecordID
WHERE BorrowingRecords.BorrowedDate >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH)  
GROUP BY Users.Department;
"""
conn.execute(department_borrowing_report)
results = conn.fetchall()

if not results:
    print("No borrowing records found for the last 6 months.")
else:
    # Print the report
    print(f"{'Department':<25}{'Total Borrowed':<15}{'Total Fines':<15}")
    print("-" * 55)
    for row in results:
        department = row[0]
        total_borrowed = row[1]
        total_fines = row[2] if row[2] is not None else 0  
        print(f"{department:<25}{total_borrowed:<15}{total_fines:<15}")

No borrowing records found for the last 6 months.


### Implement a search function

In [41]:
keyword = 'AI'  
format_type = 'Physical'  
start_date = '2020-01-01'  
end_date = '2024-12-31' 

keyword_search = """
SELECT Resources.Title, Resources.PublicationDate, Resources.Format
FROM Resources
JOIN ResourceKeywords ON Resources.ResourceID = ResourceKeywords.ResourceID
JOIN Keywords ON ResourceKeywords.KeywordID = Keywords.KeywordID
WHERE Keywords.KeywordText LIKE %s  
AND Resources.Format = %s  
AND Resources.PublicationDate BETWEEN %s AND %s;  
"""

conn.execute(keyword_search, (f'%{keyword}%', format_type, start_date, end_date))
results = conn.fetchall()

if results:
    for row in results:
        print(f"Title: {row[0]}, Publication Date: {row[1]}, Format: {row[2]}")
else:
    print("No results found.")

Title: Intro to AI, Publication Date: 2023-01-01, Format: Physical


### Resource Recommendation Procedure

In [42]:
user_id = 1

resource_recommendation = """
SELECT R.Title
FROM BorrowingRecords BR
JOIN Resources R ON BR.ResourceID = R.ResourceID
WHERE BR.UserID = %s
GROUP BY R.ResourceID
ORDER BY COUNT(BR.BorrowingRecordID) DESC;
"""

conn.execute(resource_recommendation, (user_id,))
results = conn.fetchall()

if results:
    print("Recommended Resources:")
    for row in results:
        print(f"Title: {row[0]}")
else:
    print("No recommendations found.")

Recommended Resources:
Title: Intro to AI
Title: Machine Learning Basics


## Views

### Available Resources View

In [ ]:
available_resources_sql = """
CREATE OR REPLACE VIEW AvailableResources AS
SELECT R.ResourceID, R.Title, R.Format
FROM Resources R
LEFT JOIN BorrowingRecords BR ON R.ResourceID = BR.ResourceID
GROUP BY R.ResourceID
HAVING COUNT(BR.BorrowingRecordID) = 0 OR R.Format = 'Digital';
"""

conn.execute(available_resources_sql)

print("Results for AvailableResources:")
conn.execute("SELECT * FROM AvailableResources;")
results = conn.fetchall()
if results:
    for row in results:
        print(row)
else:
    print("No results found.")
print()

Results for AvailableResources:
(2, 'Machine Learning Basics', 'Digital')
(6, 'The Martian Chronicles', 'Digital')



### User Borrowing Activity View

In [44]:
user_borrowing_activity_sql = """
CREATE OR REPLACE VIEW UserBorrowingActivity AS
SELECT U.UserID, U.Name, U.Department,
       COUNT(BR.BorrowingRecordID) AS TotalBorrows,
       SUM(Fines.FineAmount) AS TotalFines
FROM Users U
LEFT JOIN BorrowingRecords BR ON U.UserID = BR.UserID
LEFT JOIN Fines ON BR.BorrowingRecordID = Fines.BorrowingRecordID
GROUP BY U.UserID;
"""

conn.execute(user_borrowing_activity_sql)

print("Results for UserBorrowingActivity:")
conn.execute("SELECT * FROM UserBorrowingActivity;")
results = conn.fetchall()
if results:
    for row in results:
        print(row)
else:
    print("No results found.")
print()

Results for UserBorrowingActivity:
(1, 'Alice Student', 'Computer Science', 2, Decimal('10.00'))
(2, 'Bob Faculty', 'Engineering', 2, Decimal('5.00'))
(3, 'Charlie Staff', 'Administration', 1, None)



### Top-Rated Resources View

In [45]:
top_rated_resources_sql = """
CREATE OR REPLACE VIEW TopRatedResources AS
SELECT R.ResourceID, R.Title, AVG(Rev.Rating) AS AverageRating
FROM Resources R
JOIN Reviews Rev ON R.ResourceID = Rev.ResourceID
GROUP BY R.ResourceID
HAVING AVG(Rev.Rating) IS NOT NULL
ORDER BY AverageRating DESC;
"""

# Execute the SQL command
conn.execute(top_rated_resources_sql)

# Print results for Top-Rated Resources View
print("Results for TopRatedResources:")
conn.execute("SELECT * FROM TopRatedResources;")
results = conn.fetchall()
if results:
    for row in results:
        print(row)
else:
    print("No results found.")
print()

Results for TopRatedResources:
(4, "Shakespeare's Sonnets", Decimal('5.0000'))
(1, 'Intro to AI', Decimal('5.0000'))
(5, "Shakespeare's Sonnets", Decimal('4.0000'))
(2, 'Machine Learning Basics', Decimal('4.0000'))
(3, 'Calculus for Dummies', Decimal('3.0000'))
(6, 'The Martian Chronicles', Decimal('2.0000'))



### User Reservation Status View

In [46]:
user_reservation_status_sql = """
CREATE OR REPLACE VIEW UserReservationStatus AS
SELECT U.UserID, U.Name, R.Title, Res.Status
FROM Users U
JOIN Reservations Res ON U.UserID = Res.UserID
JOIN Resources R ON Res.ResourceID = R.ResourceID;
"""

conn.execute(user_reservation_status_sql)

print("Results for UserReservationStatus:")
conn.execute("SELECT * FROM UserReservationStatus;")
results = conn.fetchall()
if results:
    for row in results:
        print(row)
else:
    print("No results found.")
print()

Results for UserReservationStatus:
(1, 'Alice Student', 'Intro to AI', 'Approved')
(1, 'Alice Student', "Shakespeare's Sonnets", 'Approved')
(2, 'Bob Faculty', 'Machine Learning Basics', 'Pending')
(2, 'Bob Faculty', "Shakespeare's Sonnets", 'Approved')
(3, 'Charlie Staff', 'Calculus for Dummies', 'Denied')
(3, 'Charlie Staff', 'The Martian Chronicles', 'Pending')



### Resource Borrowing History View

In [47]:
resource_borrowing_history_sql = """
CREATE OR REPLACE VIEW ResourceBorrowingHistory AS
SELECT R.Title, U.Name, BR.BorrowedDate, BR.DueDate, BR.ReturnedDate
FROM BorrowingRecords BR
JOIN Resources R ON BR.ResourceID = R.ResourceID
JOIN Users U ON BR.UserID = U.UserID;
"""

conn.execute(resource_borrowing_history_sql)

print("Results for ResourceBorrowingHistory:")
conn.execute("SELECT * FROM ResourceBorrowingHistory;")
results = conn.fetchall()
if results:
    for row in results:
        print(row)
else:
    print("No results found.")
print()

Results for ResourceBorrowingHistory:
('Intro to AI', 'Alice Student', datetime.date(2024, 1, 15), datetime.date(2024, 2, 15), None)
('Machine Learning Basics', 'Alice Student', datetime.date(2024, 1, 20), datetime.date(2024, 2, 20), datetime.date(2024, 2, 10))
('Calculus for Dummies', 'Bob Faculty', datetime.date(2024, 1, 18), datetime.date(2024, 2, 18), None)
("Shakespeare's Sonnets", 'Bob Faculty', datetime.date(2024, 1, 30), datetime.date(2024, 2, 28), datetime.date(2024, 2, 27))
("Shakespeare's Sonnets", 'Charlie Staff', datetime.date(2024, 1, 25), datetime.date(2024, 2, 25), None)



## Procedures

##### Parameter Prefix (p_). So, any variable that has this prefix is included in the procedure

### Overdue Fine Calculation Procedure

In [48]:
create_procedure_fine_calculation = """
CREATE PROCEDURE CalculateOverdueFines()
BEGIN
    -- Declaring variables
    DECLARE record_id INT;
    DECLARE overdue_days INT;
    DECLARE fine_amount DECIMAL(10, 2);
    DECLARE resource_format ENUM('Physical', 'Digital');

    -- Declaring a cursor to iterate through borrowing records that are overdue
    DECLARE cur CURSOR FOR 
        SELECT BR.BorrowingRecordID, DATEDIFF(CURDATE(), BR.DueDate) AS overdue_days, R.Format 
        FROM BorrowingRecords BR
        JOIN Resources R ON BR.ResourceID = R.ResourceID
        WHERE BR.ReturnedDate IS NULL AND BR.DueDate < CURDATE();

    -- Declaring a handler to continue the loop if no more records are found
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET record_id = NULL;

    -- Open the cursor for reading
    OPEN cur;

    -- Loop through each borrowing record in the cursor
    read_loop: LOOP
        -- Fetch the next record into variables
        FETCH cur INTO record_id, overdue_days, resource_format;

        -- If no record is found, exit the loop
        IF record_id IS NULL THEN
            LEAVE read_loop;
        END IF;

        -- Calculate the fine based on resource format and overdue days
        SET fine_amount = CASE 
            WHEN resource_format = 'Physical' THEN overdue_days * 1.00  -- $1 per day for physical resources
            WHEN resource_format = 'Digital' THEN overdue_days * 0.50   -- $0.50 per day for digital resources
            ELSE 0  -- Default case, no fine
        END;

        -- Insert the calculated fine into the Fines table
        INSERT INTO Fines (BorrowingRecordID, FineAmount, PaymentDate) 
        VALUES (record_id, fine_amount, NULL);
        
        -- Print the calculated fine for verification
        SELECT CONCAT('Calculated fine for BorrowingRecordID ', record_id, ': ', fine_amount) AS OutputMessage;
    END LOOP;

    -- Close the cursor after processing all records
    CLOSE cur;
END;
"""




# Execute the procedure creation
conn.execute(create_procedure_fine_calculation)
mydb.commit()


In [49]:
conn.execute("CALL CalculateOverdueFines()")
fine_results = conn.fetchall()
for result in fine_results:
    print(result)

('Calculated fine for BorrowingRecordID 1: 252.00',)


### User Account Creation Procedure

In [ ]:
mydb, conn = create_connection()
create_procedure_user_account = """
CREATE PROCEDURE CreateUserAccount(
    IN p_name VARCHAR(255), 
    IN p_email VARCHAR(255), 
    IN p_department VARCHAR(255), 
    IN p_user_type VARCHAR(50)
)
BEGIN
    -- Declaring a variable to hold the UserTypeID based on the provided user type
    DECLARE userTypeId INT;

    -- Select the UserTypeID from UserTypes based on the provided user type name
    SELECT UserTypeID INTO userTypeId 
    FROM UserTypes 
    WHERE TypeName = p_user_type LIMIT 1;

    -- Insert a new user into the Users table with the provided details and the retrieved UserTypeID
    INSERT INTO Users (Name, Email, Department, UserTypeID) 
    VALUES (p_name, p_email, p_department, userTypeId);
    
    -- Print a message indicating the account has been created
    SELECT CONCAT('User account created for: ', p_name) AS OutputMessage;
END;
"""

# Execute the procedure creation
conn.execute(create_procedure_user_account)
mydb.commit()

Connection to the database was successful.


In [51]:
name = 'John Doe'
email = 'john.doe@example.com'
department = 'Engineering'
user_type = 'Student'  

call_procedure = f"CALL CreateUserAccount('{name}', '{email}', '{department}', '{user_type}');"
conn.execute(call_procedure)

result = conn.fetchall()
if result:
    for row in result:
        print("\n",row[0])  
else:
    print("No output message.")


 User account created for: John Doe


### Resource Reservation Management Procedure

In [ ]:
mydb, conn = create_connection()
create_procedure_reservation_management = """
CREATE PROCEDURE ManageResourceReservation(
    IN p_user_id INT, 
    IN p_resource_id INT, 
    IN p_status ENUM('Pending', 'Approved', 'Denied')
)
BEGIN
    -- Declaring a variable to check if the resource is available for reservation
    DECLARE resource_available INT;

    -- Count the number of approved reservations for the specified resource
    SELECT COUNT(*) INTO resource_available 
    FROM Reservations 
    WHERE ResourceID = p_resource_id AND Status = 'Approved';

    -- If there are no approved reservations, insert a new reservation
    IF resource_available = 0 THEN
        INSERT INTO Reservations (UserID, ResourceID, ReservationDate, Status) 
        VALUES (p_user_id, p_resource_id, CURDATE(), p_status);
        
        -- Print a message indicating the reservation has been made
        SELECT CONCAT('Reservation created for User ID: ', p_user_id, ' on Resource ID: ', p_resource_id) AS OutputMessage;
    ELSE
        -- If resource is already reserved, update the status of the existing reservation
        UPDATE Reservations 
        SET Status = p_status 
        WHERE UserID = p_user_id AND ResourceID = p_resource_id AND Status = 'Pending';

        -- Print a message indicating the reservation status has been updated
        SELECT CONCAT('Reservation status updated for User ID: ', p_user_id, ' on Resource ID: ', p_resource_id) AS OutputMessage;
    END IF;
END;
"""

# Execute the procedure creation
conn.execute(create_procedure_reservation_management)
mydb.commit()


Connection to the database was successful.


In [53]:
user_id = 1           
resource_id = 1      
status = 'Pending'    # Status of the reservation (Pending, Approved, Denied)

call_procedure = f"CALL ManageResourceReservation({user_id}, {resource_id}, '{status}');"
conn.execute(call_procedure)

result = conn.fetchall()
if result:
    for row in result:
        print("\n",row[0])  
else:
    print("No output message.")



 Reservation status updated for User ID: 1 on Resource ID: 1


### Resource Return Procedure

In [54]:
mydb, conn = create_connection()
create_procedure_resource_return = """
CREATE PROCEDURE ReturnResource(
    IN p_borrowing_record_id INT  -- The ID of the borrowing record to return the resource
)
BEGIN
    -- Declaring variables to store the due date and fine amount
    DECLARE due_date DATE;
    DECLARE fine_amount DECIMAL(10, 2);

    -- Get the due date for the specified borrowing record
    SELECT DueDate INTO due_date 
    FROM BorrowingRecords 
    WHERE BorrowingRecordID = p_borrowing_record_id;

    -- Check if the current date is past the due date
    IF CURDATE() > due_date THEN
        -- Calculate the fine based on the number of overdue days
        SET fine_amount = DATEDIFF(CURDATE(), due_date) * 1.00;

        -- Insert the fine record into the Fines table
        INSERT INTO Fines (BorrowingRecordID, FineAmount, PaymentDate) 
        VALUES (p_borrowing_record_id, fine_amount, CURDATE());
        
        -- Print a message indicating the fine has been added
        SELECT CONCAT('Fine of ', fine_amount, ' added for Borrowing Record ID: ', p_borrowing_record_id) AS OutputMessage;
    END IF;

    -- Update the borrowing record to set the returned date
    UPDATE BorrowingRecords 
    SET ReturnedDate = CURDATE() 
    WHERE BorrowingRecordID = p_borrowing_record_id;

    -- Print a message indicating the resource has been returned
    SELECT CONCAT('Resource returned for Borrowing Record ID: ', p_borrowing_record_id) AS OutputMessage;
END;
"""

# Execute the procedure creation
conn.execute(create_procedure_resource_return)
mydb.commit()


Connection to the database was successful.


In [55]:
mydb, conn = create_connection()
borrowing_record_id = 3  # The ID of the borrowing record to return

call_procedure = f"\nCALL ReturnResource({borrowing_record_id});"
conn.execute(call_procedure)

result = conn.fetchall()
if result:
    for row in result:
        print("\n",row[0]) 
else:
    print("No output message.")

Connection to the database was successful.

 Fine of 249.00 added for Borrowing Record ID: 3


### Resource Recommendation Procedure (Optional)

In [56]:
mydb, conn = create_connection()
create_procedure_resource_recommendation = """
CREATE PROCEDURE RecommendResources(
    IN p_user_id INT  -- The ID of the user for whom we are recommending resources
)
BEGIN
    -- Selecting the resources that the user has borrowed, ordered by the count of times borrowed (most borrowed first)
    SELECT R.Title, COUNT(BR.BorrowingRecordID) AS TimesBorrowed
    FROM BorrowingRecords BR
    JOIN Resources R ON BR.ResourceID = R.ResourceID
    WHERE BR.UserID = p_user_id  -- Filter for the specific user
    GROUP BY R.ResourceID  -- Group by resource to count how many times each has been borrowed
    ORDER BY TimesBorrowed DESC  -- Order by the count of borrowings, descending
    LIMIT 5;  -- Limit the recommendations to the top 5 most borrowed resources

    -- Print a message indicating the recommendation has been generated
    SELECT CONCAT('Top resource recommendations generated for User ID: ', p_user_id) AS OutputMessage;
END;
"""

# Execute the procedure creation
conn.execute(create_procedure_resource_recommendation)
mydb.commit()


Connection to the database was successful.


In [57]:
mydb, conn = create_connection()
user_id = 1  # Replace this with the actual user ID

call_procedure = f"CALL RecommendResources({user_id});"
conn.execute(call_procedure)

results = conn.fetchall()
if results:
    for row in results:
        print(f"Recommended Resource: {row[0]}, Times Borrowed: {row[1]}")
else:
    print("No recommendations found.")


Connection to the database was successful.
Recommended Resource: Intro to AI, Times Borrowed: 1
Recommended Resource: Machine Learning Basics, Times Borrowed: 1


# Disclosure Statement

#### I have used ChatGPT, an AI language model developed by OpenAI, to assist in generating dummy data and to facilitate the commenting on the code for this project. The AI was utilized to streamline certain tasks and improve the overall clarity and structure of the code but was not responsible for the project's core design or logic. All final decisions and implementations were made independently.